In [1]:
import gensim
from xlrd import open_workbook
import os
from nltk.tokenize import word_tokenize, sent_tokenize
import json
import pandas as pd
from nltk.corpus import stopwords

import nltk
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from os import listdir, makedirs
from os.path import isfile, join, exists
import os
from nltk.corpus import stopwords
from xlrd import open_workbook
import json
from gensim import similarities
import numpy as np

In [2]:
        
unspec_df = pd.read_excel('ignore/UNSPSC English v220601 project.xlsx','Sheet1', index_col=None, na_values=['NA'])
unspec = unspec_df.drop(['Family Definition','Family','Family Title','Segment Title','Segment Definition','Segment','Version','Key','Synonym','Acronym'],axis = 1)
cols = ['Class Title','Class Definition']
cols2 =['Commodity Title','Commodity Definition']
unspec["class"] = unspec[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
unspec["commodity"] = unspec[cols2].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
#unspec["commodity"]=[row.replace(' ',',') for row in unspec["commodity"]]
#unspec["class"]=[row.replace(' ',',') for row in unspec["class"]]
unspec2 = unspec.drop(['Class','Class Title','Class Definition','Commodity Title','Commodity Definition'],axis=1)
unspec2=unspec2.rename(columns={"Commodity":"level_1"})
nums = unspec2['level_1'].tolist()

In [3]:
county_df = pd.read_excel('ignore/eCAPS_COMM_11072019.xlsx','COMM_CLS', index_col=None, na_values=['NA'])
county_df2 = pd.read_excel('ignore/eCAPS_COMM_11072019.xlsx','COMM_ITM', index_col=None, na_values=['NA'])
county_df.drop(['COMM_GP','COMM_DET','ACT_INACT_FL','NAICS_CD'],axis=1)
county_df2.drop(['COMM_GP','COMM_DET','ACT_INACT_FL','NAICS_CD'],axis=1)
countyresult = pd.merge(county_df,county_df2,on='COMM_CLS')
cr = countyresult.drop(['DSCR_EXT_y','DSCR_EXT_DV_x','DSCR_EXT_x','COMM_DSCR_UP_x','COMM_CD_x','COMM_ITM_x','COMM_GP_x','COMM_DET_x','ACT_INACT_FL_x','NAICS_CD_x','COMM_DET_y','COMM_GP_y','COMM_DET_y','ACT_INACT_FL_y','NAICS_CD_y','COMM_DSCR_UP_y','DSCR_EXT_DV_y'],axis=1)
cr2 = cr.rename(columns={"KEYWD_x":"class","KEYWD_y":"commodity","COMM_CD_y":"level_1"})
#cr2["commodity"]=[row.replace(' ',',') for row in cr2["commodity"]]
#cr2["class"]=[row.replace(' ',',') for row in cr2["class"]]


In [4]:

df1= cr2[['class','commodity','level_1']]
df2 = df1.append(unspec2,ignore_index=True,sort=False)


In [5]:
df2.head(15)

,class,commodity,level_1
0,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...",RADIO-NAVIGATION-AIRCRAFT,3510
1,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...",RADAR-AIRCRAFT,3515
2,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...",AIRPLANES,3520
3,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...",AUTOMATIC PILOT SYSTEMS,3525
4,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...",BAGGAGE HANDLING EQUIPMENT AND PARTS,3526
5,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...","BEACONS, VISUAL AND RUNWAY LIGHTS",3527
6,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...",ENGINES AND PARTS-AIRPLANE,3530
7,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...","ENGINES AND PARTS, HELICOPTER",3535
8,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...","EQUIPMENT AND SUPPLIES, AIRPLANE",3540
9,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU...","EQUIPMENT AND SUPPLIES, HELICOPTER",3545


In [6]:
df9 = df2.apply(lambda x: ','.join(x.astype(str)),axis = 1)

In [7]:
df_clean2 = pd.DataFrame({'clean':df9})

In [8]:
df_clean2.head()

,clean
0,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU..."
1,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU..."
2,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU..."
3,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU..."
4,"AIRCRAFT AND AIRPORT, EQUIPMENT, PARTS, AND SU..."


In [9]:
sent23 = [row.split(',') for row in df_clean2['clean']]

In [10]:
print(sent23[:5])

[['AIRCRAFT AND AIRPORT', ' EQUIPMENT', ' PARTS', ' AND SUPPLIES', 'RADIO-NAVIGATION-AIRCRAFT', '3510'], ['AIRCRAFT AND AIRPORT', ' EQUIPMENT', ' PARTS', ' AND SUPPLIES', 'RADAR-AIRCRAFT', '3515'], ['AIRCRAFT AND AIRPORT', ' EQUIPMENT', ' PARTS', ' AND SUPPLIES', 'AIRPLANES', '3520'], ['AIRCRAFT AND AIRPORT', ' EQUIPMENT', ' PARTS', ' AND SUPPLIES', 'AUTOMATIC PILOT SYSTEMS', '3525'], ['AIRCRAFT AND AIRPORT', ' EQUIPMENT', ' PARTS', ' AND SUPPLIES', 'BAGGAGE HANDLING EQUIPMENT AND PARTS', '3526']]


In [11]:
print(len(sent23))

163115


In [12]:
model2 = gensim.models.word2vec.Word2Vec(sent23,min_count=1,size=300, workers=6, window= 4,cbow_mean=1)

In [13]:
print(len(model2.wv.vocab))

466078


In [14]:
model2.wv.__getitem__('39101601')

array([-2.80980999e-03,  1.31541258e-03, -1.72519800e-03,  3.51986685e-03,
        4.36890405e-03,  2.85443943e-03,  6.72989758e-04, -1.47552625e-03,
       -2.68949056e-03, -3.43843061e-03,  1.52570661e-03,  2.52473727e-03,
        4.53344034e-03, -3.35490680e-03, -3.65078379e-03,  4.80535557e-04,
       -4.79909009e-04, -4.08498629e-04,  9.17964848e-04, -2.01070891e-03,
       -2.77642277e-03, -4.75544715e-04, -2.44910107e-03, -3.88924917e-03,
        5.22434129e-04, -4.02551005e-03,  1.45596627e-03, -2.17442214e-03,
        6.62889041e-04, -1.51107821e-03, -2.90205423e-03,  2.43105390e-03,
        3.79093899e-03,  2.98959319e-03, -3.66297038e-03,  2.14953441e-03,
        2.89493031e-03, -3.20550578e-04, -1.74015830e-03,  1.09569659e-03,
       -8.24619900e-04, -1.98771781e-03, -1.30095263e-03, -5.07510267e-05,
        8.23652590e-06,  2.35917838e-03,  3.17131239e-03,  1.06536679e-03,
       -3.16652772e-03, -2.16776156e-04, -3.10744718e-03, -1.14115013e-03,
        4.69409674e-03, -

In [23]:
def NumberofDocs(li,nums):
    shortlist=[]
    for i in li:
        if(i!='nan'):
            point = model2.wv.similarity(str(i),nums)
            if(point>.9000001):
                shortlist.append(i) 
    
    return shortlist

result =NumberofDocs(nums,'4045') 
print(len(result))


35863


In [22]:
result2 =NumberofDocs(nums,'3526') 
print(len(result2))
print(result2)

0
[]


In [17]:
model2.wv.similarity('39101601','39101603')

0.71096516

In [18]:
model2.wv.similarity('10101708','4045')

0.74658716

In [19]:
model2.wv.most_similar('3105')[:15]

[('Seam gauge nan', 0.8256432414054871),
 ('Window screens nan', 0.8251415491104126),
 ('Green leaf lettuce purees The variety of lettuce known as green leaf lettuce which have been pureed.',
  0.8248039484024048),
 ('85671503', 0.8246505260467529),
 ('Frozen organic papayas Succulent yellow fruit of a large plant (carica papaya) of the family caricaceae that is considered a tree',
  0.8245862722396851),
 ('  via natural or artificial opening endoscopic This classification denotes the group of activities that yields a surgical intervention or procedure to revision of autologous tissue substitute in larynx',
  0.8245682120323181),
 ('50445302', 0.8242932558059692),
 ('85632301', 0.8238834738731384),
 ('Canned or jarred organic chinese garlic stems The variety of organic garlic known as organic chinese garlic stems that has been canned or jarred for preservation.',
  0.8237274885177612),
 ('Non GMO david sun peaches The variety of non GMO peaches known as non GMO david sun peaches. This 

In [20]:
"""
book = open_workbook('UNSPSC English v220601 project.xlsx')
'''book = open_workbook('Unspec List2b.xlsx')'''
'''To work on the UNSPSC sheet you need to change the values of 0 to 12 and 1 to
16 in order to make the it work.'''
dict_list = []
sheet = book.sheet_by_index(0)
# read header values into the list
keys = [sheet.cell(0, col_index).value for col_index in range(sheet.ncols)]

for row_index in range(1, sheet.nrows):
    d = {keys[col_index]: sheet.cell(row_index, col_index).value
         for col_index in range(sheet.ncols)}
    dict_list.append(d)

print(len(dict_list))
    
doclist =[]
listOfEntry =[]
entrylist = []
df7 = pd.DataFrame()


for entry in dict_list:
    if(entry.get("Family")==""):
        continue
    elif(entry.get("Family")!="" and entry.get("Class")==""):
        if(listOfEntry!=[]):
            #print(listOfEntry)
            doclist += entrylist
            continue
    elif(entry.get("Commodity")==""):
        continue
    else:
        if(entry.get("Class Definition")!=""):
            f = str(sent_tokenize(entry.get("Class Definition").lower())).strip('[]').strip('\'')
            #print(f)
        else:
            f=str([" "]).strip('[]').strip('\'')
        e = str(sent_tokenize(entry.get("Class Title").lower())).strip('[]').strip('\'')
        #print(e)
        g = str(sent_tokenize(entry.get("Commodity Title").lower())).strip('[]').strip('\'')
        #print(g)
        if(entry.get("Commodity Definition")!=""):
            h = str(sent_tokenize(entry.get("Commodity Definition").lower())).strip('[]').strip('\'')
            #print(h)
        else:
            h = str([" "]).strip('[]').strip('\'')
        i=int(entry.get("Commodity"))
        df7 = df7.append({'class':e+f,'commodity':g+h,'level_1': i},ignore_index=True)
        
  
        
doclist.append(listOfEntry) 

print("done")        
"""

"""
model2.wv.vocab
model2.save("W2V-Model")

text = "AIR CONDITIONING, HEATING, AND VENTILATING: EQUIPMENT, PARTS  HYDRONIC SPECIALTIES"
idt =['101'] 
text = text.lower()
newEntry = []
newEn =[]
stop_words = set(stopwords.words('english'))
ch = [ '``','``', "''",',','.','\\n',"'",";",":","(",")","-","--"]


Wtest = word_tokenize(text)
Wtest = [ j for j in Wtest if not j in ch ]
Wtest = [w for w in Wtest if not w in stop_words]
newEntry.append(Wtest)

def concatenate_list_data(list):
    result= ''
    for element in list:
        result += str(element+" ")
    return result

newEntry[0] = concatenate_list_data(newEntry[0])
newEntry += idt
newEn.append(newEntry)
print(newEn)

new_mod = gensim.models.Word2Vec.load("W2V-Model")
new_mod.build_vocab(newEn,update = True)
new_mod.train(newEn,total_examples=1,epochs = 1)

new_mod.wv.similarity('30121703.0','101')
new_mod.wv.most_similar_cosmul('101')

new_mod.wv.similar_by_vector('101')

"""


'\nmodel2.wv.vocab\nmodel2.save("W2V-Model")\n\ntext = "AIR CONDITIONING, HEATING, AND VENTILATING: EQUIPMENT, PARTS  HYDRONIC SPECIALTIES"\nidt =[\'101\'] \ntext = text.lower()\nnewEntry = []\nnewEn =[]\nstop_words = set(stopwords.words(\'english\'))\nch = [ \'``\',\'``\', "\'\'",\',\',\'.\',\'\\n\',"\'",";",":","(",")","-","--"]\n\n\nWtest = word_tokenize(text)\nWtest = [ j for j in Wtest if not j in ch ]\nWtest = [w for w in Wtest if not w in stop_words]\nnewEntry.append(Wtest)\n\ndef concatenate_list_data(list):\n    result= \'\'\n    for element in list:\n        result += str(element+" ")\n    return result\n\nnewEntry[0] = concatenate_list_data(newEntry[0])\nnewEntry += idt\nnewEn.append(newEntry)\nprint(newEn)\n\nnew_mod = gensim.models.Word2Vec.load("W2V-Model")\nnew_mod.build_vocab(newEn,update = True)\nnew_mod.train(newEn,total_examples=1,epochs = 1)\n\nnew_mod.wv.similarity(\'30121703.0\',\'101\')\nnew_mod.wv.most_similar_cosmul(\'101\')\n\nnew_mod.wv.similar_by_vector(\'10